In [51]:
import pandas as pd
import os

In [31]:
import switch_processing
from importlib import reload
reload(switch_processing)

<module 'switch_processing' from '/Users/andrewtownsend/Desktop/Ribo_project_workspace/switch_processing.py'>

In [50]:
# see test_to_df.ipynb for example input file
# notably this only works with forward engineered toehold switches from green et al
xl_file = "test_to_df.xlsx"
source_df = pd.read_excel(xl_file)
results_list = []

In [54]:
import json
with open('useful_constants.json', 'r') as file:
    data = json.load(file)

In [ ]:
display(source_df)

In [53]:
for index in range(len(source_df)):
    row = source_df.iloc[index]
    switch = row["toehold_switch_sequences"]
    trigger = row["trigger_switch_sequences"]
    
    switch_results = switch_processing.evaluate_sequence(
        sequence=switch, 
        desired_structure=data["forward_engineered_toehold_switch_sec_struc"],
        unbound_region=(0, 17))
    trigger_results = switch_processing.evaluate_sequence(
        sequence=trigger,
        desired_structure=data["trigger_secondary_structure"],
        unbound_region=(29, 64)
    )
    complex_results = switch_processing.evaluate_target_switch_complex(
        target=trigger,
        switch=switch,
        desired_secondary_structure=data["trigger_switch_complex_sec_struct"]
    )

    switch_mfe_structure = switch_processing.get_mfe_secondary_structure(switch)
    trigger_mfe_structure = switch_processing.get_mfe_secondary_structure(trigger)
    trigger_switch_complex_mfe_structure = switch_processing.get_mfe_secondary_structure([trigger, switch])

    score = 5*trigger_results["l_toehold_region"] + 4*switch_results["l_toehold_region"] + 3*complex_results["normalized_complex_ensemble_defect"]

    results_list.append({
        "toehold_switch_sequence": switch,
        "trigger_switch_sequence": trigger,
        "l_switch_toehold_region": switch_results["l_toehold_region"],
        "l_trigger_toehold_region": trigger_results["l_toehold_region"],
        "normalized_ensemble_defect_switch": switch_results["sequence_normalized_ensemble_defect"],
        "normalized_ensemble_defect_trigger": trigger_results["sequence_normalized_ensemble_defect"],
        "ratio_unbound_strands_switch": switch_results["ratio_unbound_strands"],
        "ratio_unbound_strands_trigger": trigger_results["ratio_unbound_strands"],
        "normalized_complex_ensemble_defect": complex_results["normalized_complex_ensemble_defect"],
        "switch_mfe_structure": switch_mfe_structure,
        "trigger_mfe_structure": trigger_mfe_structure,
        "trigger_switch_complex_mfe_structure": trigger_switch_complex_mfe_structure,
        "green_2014_score": score
    })

    # if not os.path.exists(f"images/switch"):
    #     os.makedirs("images/switch")
    # switch_processing.draw_sequence(sequence=switch,
    #                             secondary_structure=switch_mfe_structure,
    #                             colors=data["forward_engineered_toehold_colors"],
    #                             file_name=f"images/switch/switch_{index}")
    
    # if not os.path.exists(f"images/trigger"):
    #     os.makedirs("images/trigger")
    # switch_processing.draw_sequence(sequence=trigger,
    #                             secondary_structure=trigger_mfe_structure,
    #                             colors=data["trigger_sec_struct_colors"],
    #                             file_name=f"images/trigger/trigger_{index}")
    
    if not os.path.exists(f"images/complex"):
        os.makedirs("images/complex")
    switch_processing.draw_sequence(sequence=trigger + " " + switch,
                                secondary_structure=trigger_switch_complex_mfe_structure.replace("+", " "),
                                colors=data["trigger_switch_complex_colors"],
                                file_name=f"images/complex/complex{index}")
    

In [46]:
results_df = pd.DataFrame(results_list)

In [47]:
display(results_df)

,toehold_switch_sequence,trigger_switch_sequence,l_switch_toehold_region,l_trigger_toehold_region,normalized_ensemble_defect_switch,normalized_ensemble_defect_trigger,ratio_unbound_strands_switch,ratio_unbound_strands_trigger,normalized_complex_ensemble_defect,switch_mfe_structure,trigger_mfe_structure,trigger_switch_complex_mfe_structure,green_2014_score
0,GGGUCUUAUCUUAUCUAUCUCGUUUAUCCCUGCAUACAGAAACAGA...,GGGACUGACUAUUCUGUGCAAUAGUCAGUAAAGCAGGGAUAAACGA...,0.717414,0.954776,0.153190,0.035234,0.742935,0.985725,0.090668,((((.............(((((((((((..((((((.............,...((((((((((......))))))))))....................,...((((((((((......))))))))))...((((((((((((((...,7.915538
1,GGGAGUUUGAUUACAUUGUCGUUUAGUUUAGUGAUACAUAAACAGA...,GGGACAGAUCCACUGAGGCGUGGAUCUGUGAACACUAAACUAAACG...,0.850250,0.914370,0.128632,0.052221,0.723957,0.999749,0.079640,..................((((((((((..((((((.............,...((((((((((......))))))))))....................,...((((((((((......))))))))))...((((((((((((((...,8.211767
2,GGGAUCUAUUACUACUUACCAUUGUCUUGCUCUAUACAGAAACAGA...,GGGUGAUGGGACAUUCCGAUGUCCCAUCAAUAAGAGCAAGACAAUG...,0.898812,0.884493,0.127107,0.097018,0.691236,0.984172,0.086565,..................(((((((((((.((((((.............,...(((((((((((....)))))))))))....................,...(((((((((((....)))))))))))...((((((((((((((...,8.277410
3,GGGCGAUUAUGGAUUAGAGCUCCGUUUACUGUCAUACAAGAACAGA...,GGGACGAAUUCACCCUAAUGUGAAUUCGUAAAGACAGUAAACGGAG...,0.592521,0.824489,0.230670,0.099956,0.988602,0.745908,0.091726,..((..(((.((.((...(((((((((.((((((((.............,...((((((((((......)))))))))).............(((....,...((((((((((......))))))))))...((((((((((((((...,6.767708
4,GGGUAUGUAAUUGAUUUGGCUUCUGUUAGUUUCAUACAAGAACAGA...,GGGUCCAUUCUAGGUGAUACUAGAAUGGAGCAGAAACUAACAGAAG...,0.937277,0.929222,0.132512,0.054940,0.722263,0.999949,0.114675,..................((((((((..((((((((.............,.(.((((((((((......)))))))))).)..................,.(.((((((((((......)))))))))).).((((((((((((((...,8.739244
5,GGGCUUAAUCUUACCUUCGCUUGUUCUGUUCCGAUACAGAAACAGA...,GGGACAAUCGGGACGACACUCCCGAUUGUGAACGGAACAGAACAAG...,0.773652,0.886000,0.136782,0.070332,0.723082,0.999916,0.107031,((...........))...(((((((((..(((((((.............,...((((((((((......))))))))))....................,...((((((((((......))))))))))...((((((((((((((...,7.845699
6,GGGUCACUUAAUCAUUUGUCGUCGUUUCUAUCUAUACAAGAACAGA...,GGGUCGAGUAGACAGAGCUGUCUACUCGAAUAAGAUAGAAACGACG...,0.812419,0.941283,0.134174,0.062778,0.778785,0.183983,0.085112,..................((((((((((..((((((.............,...(((((((((((....)))))))))))....................,...(((((((((((....)))))))))))...((((((((((((((...,8.211426
7,GGGACCUCUACUUACUCUCACUCUUACUUCUGCAUAGUAGAACAGA...,GGGACUGAGCUGCUAUCACGCAGCUCAGUAGAGCAGAAGUAAGAGU...,0.759044,0.946921,0.142692,0.031696,0.764952,0.999904,0.110637,.................(((((((((((..((((((.............,...((((((((((......))))))))))....................,...((((((((((......))))))))))...((((((((((((((...,8.102695
8,GGGCUUACUACUUUGACACCUGAUUCUGACACGAUAACAGAACAGA...,GGGUCAAUUACCCGUGGUAGGGUAAUUGAAAGCGUGUCAGAAUCAG...,0.842177,0.749637,0.129177,0.141887,0.779827,0.999968,0.125932,..................(((((((((.(.((((((.(...........,...((((((((((......))))))))))..((.((.......))....,...((((((((((......))))))))))...((((((((((((((...,7.494691
9,GGGAAUGGAAUGAAUGAACUGCUUGUCUUAUGUAUACAGAAACAGA...,GGGCGAAGUGUCCGUAUGAGGACACUUCGACGACAUAAGACAAGCA...,0.987471,0.865242,0.092142,0.086057,0.713002,0.995129,0.104700,..................((((((((((..((((((.............,..(((((((((((......)))))))))).)..................,...((((((((((......))))))))))...((((((((((((((...,8.590192


In [49]:
results_df.to_excel("first_14_forward_engineered.xlsx")

In [27]:
switch_processing.draw_sequence(sequence=sequence,
                                secondary_structure=sec_struc,
                                colors=data["forward_engineered_toehold_colors"],
                                file_name="test_draw_toehold")